# Imports & Definitions
```
$ pip freeze
tensorflow
scikit-image
scikit-learn
imutils
opencv-python
```

In [47]:
import tensorflow
from tensorflow import keras
import numpy
import skimage.io as scikit_io
import skimage.transform as scikit_transform
import matplotlib.pyplot as matplot_plot
import matplotlib.image as matplot_image
import sklearn.model_selection as scikit_model_selection
import os
import random
import imutils

LEARNING_RATE = 0.0001
EPOCHS = 50
BATCH_SIZE = 16

INPUT_SIZE = 128
CHANNELS = 3

# Image Loading

In [2]:
        def load_image(path):
            image = scikit_io.imread(path)
            return scikit_transform.resize(image, (INPUT_SIZE, INPUT_SIZE), anti_aliasing = True)

        def show_image(image):
            matplot_plot.imshow(image)
            matplot_plot.axis('off')

In [28]:
datasource_dir = "../image-puller/images/"
dataset_dir = "images/"
labels = ["men", "women"]

random.seed(1)
test_train_ratio = 0.2

# Copy images from source into split directories of train / test for the given label
def prepare_directory(source, label):
    for image_path in os.listdir(source):
        if (image_path.endswith(".png")):
            image_set = "test" if random.random() < test_train_ratio else "train"
            destination = os.path.join(dataset_dir, image_set, label, image_path)
            scikit_io.imsave(destination, load_image(os.path.join(source, image_path)))
def prepare(label):
    prepare_directory(os.path.join(datasource_dir, label), label)
            

# Prepare Dataset

In [196]:
# Make the directories
for split_dir in ["train/", "test/"]:
    for label in labels:
        os.makedirs(dataset_dir + split_dir + label + "/", exist_ok = True)

# Copy the images over
prepare("men")
prepare("women")


# Compile the images into training, testing and validation sets

In [29]:
def compile(images):
    dataset = numpy.ndarray((len(images), INPUT_SIZE, INPUT_SIZE, CHANNELS), dtype = numpy.uint8)
    labels = numpy.ndarray((len(images)), dtype = numpy.uint8)
    for index, image in enumerate(images):
        image_data = scikit_io.imread(image)
        # Remove greyscale images
        if (len(image_data.shape) != 3):
            image_data = numpy.stack([image_data]*3, axis = -1)
        dataset[index] = image_data
        labels[index] = 0 if "women" in image else 1
    return dataset, labels


In [30]:
training_images = list(imutils.paths.list_images(dataset_dir + "train/"))
training_images_men = list(imutils.paths.list_images(dataset_dir + "train/men/"))
training_images_women = list(imutils.paths.list_images(dataset_dir + "train/women/"))
testing_images_men = list(imutils.paths.list_images(dataset_dir + "test/men/"))
testing_images_women = list(imutils.paths.list_images(dataset_dir + "test/women/"))
testing_images = training_images_men + training_images_women

In [32]:
TRAINING_SIZE_HALF = 1000
VALIDATION_SIZE_HALF = TRAINING_SIZE_HALF + 500
training_data, training_labels = compile(training_images_men[:TRAINING_SIZE_HALF] + training_images_women[:TRAINING_SIZE_HALF])
validation_data, validation_labels = compile(training_images_men[TRAINING_SIZE_HALF:VALIDATION_SIZE_HALF] + training_images_women[TRAINING_SIZE_HALF:VALIDATION_SIZE_HALF])
testing_data, testing_images = compile(testing_images)

# Flatten the images into one numpy array

In [52]:
def create_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32, 3, input_shape = [128, 128, 3], padding = "same", activation = "relu", name = "convolution_first"),
        keras.layers.Conv2D(32, 3, padding = "same", activation = "relu", name = "convolution_second"),
        keras.layers.MaxPool2D(pool_size = (2, 2), padding = "valid", name = "pooling_first"),
        keras.layers.Conv2D(64, 3, padding = "same", activation = "relu", name = "convolution_small_first"),
        keras.layers.Conv2D(64, 3, padding = "same", activation = "relu", name = "convolution_small_second"),
        keras.layers.MaxPool2D(pool_size = (2, 2), padding = "valid", name = "pooling_second"),
        keras.layers.Flatten(name = "flatten"),
        keras.layers.Dense(256, activation = "relu", name = "fully_connected_1"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(256, activation = "relu", name = "fully_connected_2"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation = "relu", name = "classification"),
        keras.layers.Activation("sigmoid")
    ])
    model.compile(loss = "binary_crossentropy", optimizer = keras.optimizers.RMSprop(learning_rate=1e-4), metrics = [ "accuracy" ])
    return model


In [53]:
barnet = create_model()
barnet.summary()
callbacks = [
    keras.callbacks.History(),
    keras.callbacks.EarlyStopping(monitor = "loss", patience = 10, verbose = 1, mode = "auto")
]

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution_first (Conv2D)   (None, 128, 128, 32)      896       
_________________________________________________________________
convolution_second (Conv2D)  (None, 128, 128, 32)      9248      
_________________________________________________________________
pooling_first (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
convolution_small_first (Con (None, 64, 64, 64)        18496     
_________________________________________________________________
convolution_small_second (Co (None, 64, 64, 64)        36928     
_________________________________________________________________
pooling_second (MaxPooling2D (None, 32, 32, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 65536)            

# TRAIN!

In [54]:
barnet.fit(training_data, training_labels, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (validation_data, validation_labels), verbose = 2, shuffle = True, callbacks = callbacks)

Epoch 1/50
125/125 - 71s - loss: 0.7743 - accuracy: 0.4980 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/50
125/125 - 70s - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
125/125 - 69s - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/50
125/125 - 69s - loss: 0.6928 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/50
125/125 - 68s - loss: 0.6928 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/50
125/125 - 78s - loss: 0.7086 - accuracy: 0.4995 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/50
125/125 - 71s - loss: 0.6941 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/50
125/125 - 70s - loss: 0.6928 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 9/50
125/125 - 70s - loss: 0.6987 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 10/50
125/125 - 69s - loss: 0.6925 - accuracy: 0.5010 - val_loss: 0.6931 - val_accura

In [51]:
test_loss, test_accuracy = barnet.evaluate(validation_data, validation_labels, verbose = 1)
print(f"Loss: {test_loss}\nAccuracy: {test_accuracy}")

32/32 [==============================] - 7s 204ms/step - loss: 0.6931 - accuracy: 0.5000
Loss: 0.6931474804878235
Accuracy: 0.5
